# Homework and bake-off: Word similarity

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Dataset readers](#Dataset-readers)
1. [Dataset comparisons](#Dataset-comparisons)
  1. [Vocab overlap](#Vocab-overlap)
  1. [Pair overlap and score correlations](#Pair-overlap-and-score-correlations)
1. [Evaluation](#Evaluation)
  1. [Dataset evaluation](#Dataset-evaluation)
  1. [Dataset error analysis](#Dataset-error-analysis)
  1. [Full evaluation](#Full-evaluation)
1. [Homework questions](#Homework-questions)
  1. [PPMI as a baseline [0.5 points]](#PPMI-as-a-baseline-[0.5-points])
  1. [Gigaword with LSA at different dimensions [0.5 points]](#Gigaword-with-LSA-at-different-dimensions-[0.5-points])
  1. [Gigaword with GloVe for a small number of iterations [0.5 points]](#Gigaword-with-GloVe-for-a-small-number-of-iterations-[0.5-points])
  1. [Dice coefficient [0.5 points]](#Dice-coefficient-[0.5-points])
  1. [t-test reweighting [2 points]](#t-test-reweighting-[2-points])
  1. [Enriching a VSM with subword information [2 points]](#Enriching-a-VSM-with-subword-information-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

Word similarity datasets have long been used to evaluate distributed representations. This notebook provides basic code for conducting such analyses with a number of datasets:

| Dataset | Pairs | Task-type | Current best Spearman $\rho$ | Best $\rho$ paper |   |
|---------|-------|-----------|------------------------------|-------------------|---|
| [WordSim-353](http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/) | 353 | Relatedness | 82.8 | [Speer et al. 2017](https://arxiv.org/abs/1612.03975) |
| [MTurk-771](http://www2.mta.ac.il/~gideon/mturk771.html) | 771 | Relatedness | 81.0 | [Speer et al. 2017](https://arxiv.org/abs/1612.03975) |
| [The MEN Test Collection](http://clic.cimec.unitn.it/~elia.bruni/MEN) | 3,000 | Relatedness | 86.6 | [Speer et al. 2017](https://arxiv.org/abs/1612.03975)  | 
| [SimVerb-3500-dev](http://people.ds.cam.ac.uk/dsg40/simverb.html) | 500 | Similarity | 61.1 | [Mrki&scaron;&cacute; et al. 2016](https://arxiv.org/pdf/1603.00892.pdf) |
| [SimVerb-3500-test](http://people.ds.cam.ac.uk/dsg40/simverb.html) | 3,000 | Similarity | 62.4 | [Mrki&scaron;&cacute; et al. 2016](https://arxiv.org/pdf/1603.00892.pdf) |

Each of the similarity datasets contains word pairs with an associated human-annotated similarity score. (We convert these to distances to align intuitively with our distance measure functions.) The evaluation code measures the distance between the word pairs in your chosen VSM (which should be a `pd.DataFrame`).

The evaluation metric for each dataset is the [Spearman correlation coefficient $\rho$](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient) between the annotated scores and your distances, as is standard in the literature. We also macro-average these correlations across the datasets for an overall summary. (In using the macro-average, we are saying that we care about all the datasets equally, even though they vary in size.)

This homework ([questions at the bottom of this notebook](#Homework-questions)) asks you to write code that uses the count matrices in `data/vsmdata` to create and evaluate some baseline models as well as an original model $M$ that you design. This accounts for 9 of the 10 points for this assignment.

For the associated bake-off, we will distribute two new word similarity or relatedness datasets and associated reader code, and you will evaluate $M$ (no additional training or tuning allowed!) on those new datasets. Systems that enter will receive the additional homework point, and systems that achieve the top score will receive an additional 0.5 points.

## Set-up

In [4]:
from collections import defaultdict
import csv
import itertools
import numpy as np
import os
import pandas as pd
from scipy.stats import spearmanr
import vsm
from IPython.display import display

In [5]:
VSM_HOME = os.path.join('data', 'vsmdata')

WORDSIM_HOME = os.path.join('data', 'wordsim')

## Dataset readers

In [6]:
def wordsim_dataset_reader(
        src_filename, 
        header=False, 
        delimiter=',', 
        score_col_index=2):
    """Basic reader that works for all similarity datasets. They are 
    all tabular-style releases where the first two columns give the 
    word and a later column (`score_col_index`) gives the score.

    Parameters
    ----------
    src_filename : str
        Full path to the source file.
    header : bool
        Whether `src_filename` has a header. Default: False
    delimiter : str
        Field delimiter in `src_filename`. Default: ','
    score_col_index : int
        Column containing the similarity scores Default: 2

    Yields
    ------
    (str, str, float)
       (w1, w2, score) where `score` is the negative of the similarity
       score in the file so that we are intuitively aligned with our
       distance-based code. To align with our VSMs, all the words are 
       downcased.

    """
    with open(src_filename) as f:
        reader = csv.reader(f, delimiter=delimiter)
        if header:
            next(reader)
        for row in reader:
            w1 = row[0].strip().lower()
            w2 = row[1].strip().lower()
            score = row[score_col_index]
            # Negative of scores to align intuitively with distance functions:
            score = -float(score)
            yield (w1, w2, score)

def wordsim353_reader():
    """WordSim-353: http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/"""
    src_filename = os.path.join(
        WORDSIM_HOME, 'wordsim353', 'combined.csv')
    return wordsim_dataset_reader(
        src_filename, header=True)

def mturk771_reader():
    """MTURK-771: http://www2.mta.ac.il/~gideon/mturk771.html"""
    src_filename = os.path.join(
        WORDSIM_HOME, 'MTURK-771.csv')
    return wordsim_dataset_reader(
        src_filename, header=False)

def simverb3500dev_reader():
    """SimVerb-3500: http://people.ds.cam.ac.uk/dsg40/simverb.html"""
    src_filename = os.path.join(
        WORDSIM_HOME, 'SimVerb-3500', 'SimVerb-500-dev.txt')
    return wordsim_dataset_reader(
        src_filename, delimiter="\t", header=False, score_col_index=3)

def simverb3500test_reader():
    """SimVerb-3500: http://people.ds.cam.ac.uk/dsg40/simverb.html"""
    src_filename = os.path.join(
        WORDSIM_HOME, 'SimVerb-3500', 'SimVerb-3000-test.txt')
    return wordsim_dataset_reader(
        src_filename, delimiter="\t", header=False, score_col_index=3)

def men_reader():
    """MEN: http://clic.cimec.unitn.it/~elia.bruni/MEN"""
    src_filename = os.path.join(
        WORDSIM_HOME, 'MEN', 'MEN_dataset_natural_form_full')
    return wordsim_dataset_reader(
        src_filename, header=False, delimiter=' ') 

This collection of readers will be useful for flexible evaluations:

In [7]:
READERS = (wordsim353_reader, mturk771_reader, simverb3500dev_reader, 
           simverb3500test_reader, men_reader)

## Dataset comparisons

This section does some basic analysis of the datasets. The goal is to obtain a deeper understanding of what problem we're solving – what strengths and weaknesses the datasets have and how they relate to each other. For a full-fledged project, we would want to continue work like this and report on it in the paper, to provide context for the results.

In [8]:
def get_reader_name(reader):
    """Return a cleaned-up name for the similarity dataset 
    iterator `reader`
    """
    return reader.__name__.replace("_reader", "")

### Vocab overlap

How many vocabulary items are shared across the datasets?

In [9]:
def get_reader_vocab(reader):
    """Return the set of words (str) in `reader`."""
    vocab = set()
    for w1, w2, _ in reader():
        vocab.add(w1)
        vocab.add(w2)
    return vocab

In [10]:
def get_reader_vocab_overlap(readers=READERS):
    """Get data on the vocab-level relationships between pairs of 
    readers. Returns a a pd.DataFrame containing this information.
    """
    data = []
    for r1, r2 in itertools.product(readers, repeat=2):       
        v1 = get_reader_vocab(r1)
        v2 = get_reader_vocab(r2)
        d = {
            'd1': get_reader_name(r1),
            'd2': get_reader_name(r2),
            'overlap': len(v1 & v2), 
            'union': len(v1 | v2),
            'd1_size': len(v1),
            'd2_size': len(v2)}
        data.append(d)
    return pd.DataFrame(data)

In [12]:
vocab_overlap = get_reader_vocab_overlap()

In [13]:
def vocab_overlap_crosstab(vocab_overlap):
    """Return an intuitively formatted `pd.DataFrame` giving 
    vocab-overlap counts for all the datasets represented in 
    `vocab_overlap`, the output of `get_reader_vocab_overlap`.
    """        
    xtab = pd.crosstab(
        vocab_overlap['d1'], 
        vocab_overlap['d2'], 
        values=vocab_overlap['overlap'], 
        aggfunc=np.mean)
    # Blank out the upper right to reduce visual clutter:
    for i in range(0, xtab.shape[0]):
        for j in range(i+1, xtab.shape[1]):
            xtab.iloc[i, j] = ''        
    return xtab        

In [14]:
vocab_overlap_crosstab(vocab_overlap)

d2,men,mturk771,simverb3500dev,simverb3500test,wordsim353
d1,,,,,
men,751,,,,
mturk771,230,1113,,,
simverb3500dev,23,67,536,,
simverb3500test,30,94,532,823,
wordsim353,86,158,13,17,437


This looks reasonable. By design, the SimVerb dev and test sets have a lot of overlap. The other overlap numbers are pretty small, even adjusting for dataset size.

### Pair overlap and score correlations

How many word pairs are shared across datasets and, for shared pairs, what is the correlation between their scores? That is, do the datasets agree?

In [15]:
def get_reader_pairs(reader):
    """Return the set of alphabetically-sorted word (str) tuples 
    in `reader`
    """
    return {tuple(sorted([w1, w2])): score for w1, w2, score in reader()}

In [16]:
def get_reader_pair_overlap(readers=READERS):
    """Return a `pd.DataFrame` giving the number of overlapping 
    word-pairs in pairs of readers, along with the Spearman 
    correlations.
    """    
    data = []
    for r1, r2 in itertools.product(READERS, repeat=2):
        if r1.__name__ != r2.__name__:
            d1 = get_reader_pairs(r1)
            d2 = get_reader_pairs(r2)
            overlap = []
            for p, s in d1.items():
                if p in d2:
                    overlap.append([s, d2[p]])
            if overlap:
                s1, s2 = zip(*overlap)
                rho = spearmanr(s1, s2)[0]
            else:
                rho = None
            # Canonical order for the pair:
            n1, n2 = sorted([get_reader_name(r1), get_reader_name(r2)])
            d = {
                'd1': n1,
                'd2': n2,
                'pair_overlap': len(overlap),
                'rho': rho}
            data.append(d)
    df = pd.DataFrame(data)
    df = df.sort_values(['pair_overlap','d1','d2'], ascending=False)
    # Return only every other row to avoid repeats:
    return df[::2].reset_index(drop=True)

In [17]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    display(get_reader_pair_overlap())

,d1,d2,pair_overlap,rho
0,men,mturk771,11,0.592191
1,men,wordsim353,5,0.700000
2,mturk771,simverb3500test,4,0.400000
3,men,simverb3500test,2,1.000000
4,simverb3500dev,simverb3500test,1,NaN
5,simverb3500test,wordsim353,0,NaN
6,simverb3500dev,wordsim353,0,NaN
7,mturk771,wordsim353,0,NaN
8,mturk771,simverb3500dev,0,NaN
9,men,simverb3500dev,0,NaN


This looks reasonable: none of the datasets have a lot of overlapping pairs, so we don't have to worry too much about places where they give conflicting scores.

## Evaluation

This section builds up the evaluation code that you'll use for the homework and bake-off. For illustrations, I'll read in a VSM created from `data/vsmdata/giga_window5-scaled.csv.gz`:

In [18]:
giga5 = pd.read_csv(
    os.path.join(VSM_HOME, "giga_window5-scaled.csv.gz"), index_col=0)

### Dataset evaluation

In [19]:
def word_similarity_evaluation(reader, df, distfunc=vsm.cosine):
    """Word-similarity evalution framework.
    
    Parameters
    ----------
    reader : iterator
        A reader for a word-similarity dataset. Just has to yield
        tuples (word1, word2, score).    
    df : pd.DataFrame
        The VSM being evaluated.        
    distfunc : function mapping vector pairs to floats.
        The measure of distance between vectors. Can also be 
        `vsm.euclidean`, `vsm.matching`, `vsm.jaccard`, as well as 
        any other float-valued function on pairs of vectors.    
        
    Raises
    ------
    ValueError
        If `df.index` is not a subset of the words in `reader`.
    
    Returns
    -------
    float, data
        `float` is the Spearman rank correlation coefficient between 
        the dataset scores and the similarity values obtained from 
        `df` using  `distfunc`. This evaluation is sensitive only to 
        rankings, not to absolute values.  `data` is a `pd.DataFrame` 
        with columns['word1', 'word2', 'score', 'distance'].
        
    """
    data = []
    for w1, w2, score in reader():
        d = {'word1': w1, 'word2': w2, 'score': score}
        for w in [w1, w2]:
            if w not in df.index:
                raise ValueError(
                    "Word '{}' is in the similarity dataset {} but not in the "
                    "DataFrame, making this evaluation ill-defined. Please "
                    "switch to a DataFrame with an appropriate vocabulary.".
                    format(w, get_reader_name(reader))) 
        d['distance'] = distfunc(df.loc[w1], df.loc[w2])
        data.append(d)
    data = pd.DataFrame(data)
    rho, pvalue = spearmanr(data['score'].values, data['distance'].values)
    return rho, data

In [20]:
rho, eval_df = word_similarity_evaluation(men_reader, giga5)

In [21]:
rho

0.40375964105441753

In [22]:
eval_df.head()

,word1,word2,score,distance
0,sun,sunlight,-50.0,0.956828
1,automobile,car,-50.0,0.979143
2,river,water,-49.0,0.970105
3,stairs,staircase,-49.0,0.980475
4,morning,sunrise,-49.0,0.963624


### Dataset error analysis

For error analysis, we can look at the words with the largest delta between the gold score and the distance value in our VSM. We do these comparisons based on ranks, just as with our primary metric (Spearman $\rho$), and we normalize both rankings so that they have a comparable number of levels.

In [23]:
def word_similarity_error_analysis(eval_df):    
    eval_df['distance_rank'] = _normalized_ranking(eval_df['distance'])
    eval_df['score_rank'] = _normalized_ranking(eval_df['score'])
    eval_df['error'] =  abs(eval_df['distance_rank'] - eval_df['score_rank'])
    return eval_df.sort_values('error')
    
    
def _normalized_ranking(series):
    ranks = series.rank(method='dense')
    return ranks / ranks.sum()    

Best predictions:

In [24]:
word_similarity_error_analysis(eval_df).head()

,word1,word2,score,distance,distance_rank,score_rank,error
1041,hummingbird,pelican,-32.0,0.975007,0.000243,0.000244,2.434543e-07
2315,lily,pigs,-13.0,0.980834,0.000488,0.000487,4.016842e-07
2951,bucket,girls,-4.0,0.983473,0.000602,0.000603,4.151568e-07
150,night,sunset,-43.0,0.968690,0.000102,0.000103,6.520315e-07
2062,oak,petals,-17.0,0.979721,0.000435,0.000436,7.162632e-07


Worst predictions:

In [25]:
word_similarity_error_analysis(eval_df).tail()

,word1,word2,score,distance,distance_rank,score_rank,error
67,branch,twigs,-45.0,0.984622,0.000630,0.000077,0.000553
190,birds,stork,-43.0,0.987704,0.000657,0.000103,0.000554
185,bloom,tulip,-43.0,0.990993,0.000663,0.000103,0.000561
167,bloom,blossom,-43.0,0.991760,0.000664,0.000103,0.000561
198,bloom,rose,-43.0,0.992406,0.000664,0.000103,0.000561


### Full evaluation

A full evaluation is just a loop over all the readers on which one want to evaluate, with a macro-average at the end:

In [26]:
def full_word_similarity_evaluation(df, readers=READERS, distfunc=vsm.cosine):
    """Evaluate a VSM against all datasets in `readers`.
    
    Parameters
    ----------
    df : pd.DataFrame
    readers : tuple 
        The similarity dataset readers on which to evaluate.
    distfunc : function mapping vector pairs to floats.
        The measure of distance between vectors. Can also be 
        `vsm.euclidean`, `vsm.matching`, `vsm.jaccard`, as well as 
        any other float-valued function on pairs of vectors.    
    
    Returns
    -------
    pd.Series
        Mapping dataset names to Spearman r values.
        
    """        
    scores = {}     
    for reader in readers:
        score, data_df = word_similarity_evaluation(reader, df, distfunc=distfunc)
        scores[get_reader_name(reader)] = score
    series = pd.Series(scores, name='Spearman r')
    series['Macro-average'] = series.mean()
    return series

In [27]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    display(full_word_similarity_evaluation(giga5))

wordsim353         0.327831
mturk771           0.143146
simverb3500dev    -0.065020
simverb3500test   -0.066314
men                0.403760
Macro-average      0.148681
Name: Spearman r, dtype: float64

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### PPMI as a baseline [0.5 points]

The insight behind PPMI is a recurring theme in word representation learning, so it is a natural baseline for our task. For this question, write a function called `run_giga_ppmi_baseline` that does the following:

1. Reads the Gigaword count matrix with a window of 20 and a flat scaling function into a `pd.DataFrame`s, as is done in the VSM notebooks. The file is `data/vsmdata/giga_window20-flat.csv.gz`, and the VSM notebooks provide examples of the needed code.

1. Reweights this count matrix with PPMI.

1. Evaluates this reweighted matrix using `full_word_similarity_evaluation`. The return value of `run_giga_ppmi_baseline` should be the return value of this call to `full_word_similarity_evaluation`.

The goal of this question is to help you get more familiar with the code in `vsm` and the function `full_word_similarity_evaluation`.

The function `test_run_giga_ppmi_baseline` can be used to test that you've implemented this specification correctly.

In [28]:
def run_giga_ppmi_baseline():
    
    ##### YOUR CODE HERE
    giga20 = pd.read_csv(
        os.path.join(VSM_HOME, "giga_window20-flat.csv.gz"), index_col=0)
    #import pdb; pdb.set_trace()
    giga20_ppmi = vsm.pmi(giga20, positive=True)
    reweighted = giga20_ppmi
    # def full_word_similarity_evaluation(df, readers=READERS, distfunc=vsm.cosine)
    result = full_word_similarity_evaluation(reweighted)
    ws_result = result.loc['wordsim353'].round(2)
    #print(ws_result)
    return result
    """print(giga20.loc['wordsim353'])
    return giga20
    arr = giga20
    arr = giga20.to_numpy()
    #row_sum = arr.sum(axis=1)
    #col_sum = arr.sum(axis=0)
    rowsum = np.sum(arr, axis=1)
    colsum = np.sum(arr, axis=0)
    sumX = np.sum(rowsum, axis=0)
    expected = np.outer(rowsum, colsum)/sumX
    oe = arr/expected
    oe[oe == 0] = 1
    pmi = np.log(oe)
    #ppmi[ppmi < 0] = 0
    #giga20[:]=pmi
    return giga20"""




In [29]:
def test_run_giga_ppmi_baseline(run_giga_ppmi_baseline):
    result = run_giga_ppmi_baseline()
    ws_result = result.loc['wordsim353'].round(2)
    ws_expected = 0.58
    assert ws_result == ws_expected, \
        "Expected wordsim353 value of {}; got {}".format(ws_expected, ws_result)

In [30]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_giga_ppmi_baseline(run_giga_ppmi_baseline)

### Gigaword with LSA at different dimensions [0.5 points]

We might expect PPMI and LSA to form a solid pipeline that combines the strengths of PPMI with those of dimensionality reduction. However, LSA has a hyper-parameter $k$ – the dimensionality of the final representations – that will impact performance. For this problem, write a wrapper function `run_ppmi_lsa_pipeline` that does the following:

1. Takes as input a count `pd.DataFrame` and an LSA parameter `k`.
1. Reweights the count matrix with PPMI.
1. Applies LSA with dimensionality `k`.
1. Evaluates this reweighted matrix using `full_word_similarity_evaluation`. The return value of `run_ppmi_lsa_pipeline` should be the return value of this call to `full_word_similarity_evaluation`.

The goal of this question is to help you get a feel for how much LSA alone can contribute to this problem. 

The  function `test_run_ppmi_lsa_pipeline` will test your function on the count matrix in `data/vsmdata/giga_window20-flat.csv.gz`.

In [31]:
def run_ppmi_lsa_pipeline(count_df, k):
    
    ##### YOUR CODE HERE
    count_df_ppmi = vsm.pmi(count_df, positive=True)
    count_df_lsa = vsm.lsa(count_df_ppmi, k)
    result = full_word_similarity_evaluation(count_df_lsa)
    return result
     




In [32]:
def test_run_ppmi_lsa_pipeline(run_ppmi_lsa_pipeline):
    giga20 = pd.read_csv(
        os.path.join(VSM_HOME, "giga_window20-flat.csv.gz"), index_col=0)
    results = run_ppmi_lsa_pipeline(giga20, k=10)
    men_expected = 0.57
    men_result = results.loc['men'].round(2)
    assert men_result == men_expected,\
        "Expected men value of {}; got {}".format(men_expected, men_result)

In [33]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_ppmi_lsa_pipeline(run_ppmi_lsa_pipeline)

### Gigaword with GloVe for a small number of iterations [0.5 points]

Ideally, we would run GloVe for a very large number of iterations on a GPU machine to compare it against its close cousin PMI. However, we don't want this homework to cost you a lot of money or monopolize a lot of your available computing resources, so let's instead just probe GloVe a little bit to see if it has promise for our task. For this problem, write a function `run_small_glove_evals` that does the following:

1. Reads in `data/vsmdata/giga_window20-flat.csv.gz`.
1. Runs GloVe for 10, 100, and 200 iterations on `data/vsmdata/giga_window20-flat.csv.gz`, using the `mittens` implementation of `GloVe`. 
  * For all the other parameters to `mittens.GloVe` besides `max_iter`, use the package's defaults.
  * Because of the way that implementation is designed, these will have to be separate runs, but they should be relatively quick. 
1. Stores the values in a `dict` mapping each `max_iter` value to its associated 'Macro-average' score according to `full_word_similarity_evaluation`. `run_small_glove_evals`  should return this `dict`.

The trend should give you a sense for whether it is worth running GloVe for more iterations.

Some implementation notes:

* Your trained GloVe matrix `X` needs to be wrapped in a `pd.DataFrame` to work with `full_word_similarity_evaluation`. `pd.DataFrame(X, index=giga20.index)` will do the trick.

* If `glv` is your GloVe model, then running `glv.sess.close()` after each model is trained will silence warnings from TensorFlow about interactive sessions being active.

Performance will vary a lot for this function, so there is some uncertainty in the testing, but `test_run_small_glove_evals` will at least check that you wrote a function with the right general logic.

In [34]:
def run_small_glove_evals():

    from mittens import GloVe
    
    ##### YOUR CODE HERE
    giga20 = pd.read_csv(
        os.path.join(VSM_HOME, "giga_window20-flat.csv.gz"), index_col=0)
    result = {}
    iter_list = [10, 100, 200]
    #iter_list = [10] 
    for iters in iter_list:
        glv = GloVe(max_iter=iters)
        # to_numpy
        X = glv.fit(giga20.to_numpy())
        #glv.sess.close()
        df = pd.DataFrame(X, index=giga20.index)
        evaluation = full_word_similarity_evaluation(df)
        #print(evaluation)
        result[iters] = evaluation['Macro-average']
    return result



In [35]:
def test_run_small_glove_evals(run_small_glove_evals):
    data = run_small_glove_evals()
    for max_iter in (10, 100, 200):
        assert max_iter in data
        assert isinstance(data[max_iter], float)

In [36]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_small_glove_evals(run_small_glove_evals)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Iteration 10: loss: 15447376.0/home/sim/anaconda3/envs/nlu/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1752: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 100: loss: 2790806.25/home/sim/anaconda3/envs/nlu/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1752: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 200: loss: 2005082.755

### Dice coefficient [0.5 points]

Implement the Dice coefficient for real-valued vectors, as

$$
\textbf{dice}(u, v) = 
1 - \frac{
  2 \sum_{i=1}^{n}\min(u_{i}, v_{i})
}{
    \sum_{i=1}^{n} u_{i} + v_{i}
}$$
 
You can use `test_dice_implementation` below to check that your implementation is correct.

In [37]:
def test_dice_implementation(func):
    """`func` should be an implementation of `dice` as defined above."""
    X = np.array([
        [  4.,   4.,   2.,   0.],
        [  4.,  61.,   8.,  18.],
        [  2.,   8.,  10.,   0.],
        [  0.,  18.,   0.,   5.]]) 
    assert func(X[0], X[1]).round(5) == 0.80198
    assert func(X[1], X[2]).round(5) == 0.67568

In [38]:
def dice(u, v):

    
    ##### YOUR CODE HERE
    denom = np.sum(u)+np.sum(v)
    num = 0.0
    len_u = len(u)
    for i in range(len_u):
        i = int(i)
        #print(u[i], v[i])
        min_i = min(u[i],v[i])
        num = num + min_i
    
    num *= 2.0
    coeff = 1 - (num/denom)
    return coeff
    



In [39]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_dice_implementation(dice)

### t-test reweighting [2 points]



The t-test statistic can be thought of as a reweighting scheme. For a count matrix $X$, row index $i$, and column index $j$:

$$\textbf{ttest}(X, i, j) = 
\frac{
    P(X, i, j) - \big(P(X, i, *)P(X, *, j)\big)
}{
\sqrt{(P(X, i, *)P(X, *, j))}
}$$

where $P(X, i, j)$ is $X_{ij}$ divided by the total values in $X$, $P(X, i, *)$ is the sum of the values in row $i$ of $X$ divided by the total values in $X$, and $P(X, *, j)$ is the sum of the values in column $j$ of $X$ divided by the total values in $X$.

For this problem, implement this reweighting scheme. You can use `test_ttest_implementation` below to check that your implementation is correct. You do not need to use this for any evaluations, though we hope you will be curious enough to do so!

In [40]:
def test_ttest_implementation(func):
    """`func` should be an implementation of t-test reweighting as 
    defined above.
    """
    X = pd.DataFrame(np.array([
        [  4.,   4.,   2.,   0.],
        [  4.,  61.,   8.,  18.],
        [  2.,   8.,  10.,   0.],
        [  0.,  18.,   0.,   5.]]))    
    actual = np.array([
        [ 0.33056, -0.07689,  0.04321, -0.10532],
        [-0.07689,  0.03839, -0.10874,  0.07574],
        [ 0.04321, -0.10874,  0.36111, -0.14894],
        [-0.10532,  0.07574, -0.14894,  0.05767]])    
    predicted = func(X)
    assert np.array_equal(predicted.round(5), actual)

In [41]:
def ttest(df):
    
    ##### YOUR CODE HERE
    tot = df.to_numpy().sum()
    df = df/tot
    rowsum = df.sum(axis=1)
    colsum = df.sum(axis=0)
    #tot = colsum.sum()
    #tot = df.sum().sum()
    #df = df/tot
    op = np.outer(rowsum, colsum)
    numer = df - op
    denom = op**0.5
    val = numer/denom
    print(val)
    return val


In [42]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_ttest_implementation(ttest)

          0         1         2         3
0  0.330556 -0.076889  0.043212 -0.105318
1 -0.076889  0.038385 -0.108737  0.075745
2  0.043212 -0.108737  0.361111 -0.148942
3 -0.105318  0.075745 -0.148942  0.057669


### Enriching a VSM with subword information [2 points]

It might be useful to combine character-level information with word-level information. To help you begin asssessing this idea, this question asks you to write a function that modifies an existing VSM so that the representation for each word $w$ is the element-wise sum of $w$'s original word-level representation with all the representations for the n-grams $w$ contains. 

The following starter code should help you structure this and clarify the requirements, and a simple test is included below as well.

You don't need to write a lot of code; the motivation for this question is that the function you write could have practical value.

In [43]:
def subword_enrichment(df, n=4):
    
    # 1. Use `vsm.ngram_vsm` to create a character-level 
    # VSM from `df`, using the above parameter `n` to 
    # set the size of the ngrams.
    
    ##### YOUR CODE HERE
    ngram_vsm = vsm.ngram_vsm(df, n)

        
    # 2. Use `vsm.character_level_rep` to get the representation
    # for every word in `df` according to the character-level
    # VSM you created above.
    
    ##### YOUR CODE HERE
    row_count = df.shape[0]
    #char_rep_np = np.zeros(df.shape)
    char_rep_np = []
    df_np = df.to_numpy()
    for i in range(row_count):
        #char_rep_np[i][:] = vsm.character_level_rep( df.iloc[i], ngram_vsm, n)
        #char_rep = vsm.character_level_rep( df.iloc[i], ngram_vsm, n)
        #print(df.iloc[i])
                         # def character_level_rep(word, cf, n=4):
        char_rep_np.append(vsm.character_level_rep(df.index[i], ngram_vsm, n))
    char_rep_np = np.array(char_rep_np)
    df_char = pd.DataFrame(char_rep_np, index=df.index)
    # 3. For each representation created at step 2, add in its
    # original representation from `df`. (This should use
    # element-wise addition; the dimensionality of the vectors
    # will be unchanged.)
                            
    ##### YOUR CODE HERE

    df_agg = df_char + df
    
    # 4. Return a `pd.DataFrame` with the same index and column
    # values as `df`, but filled with the new representations
    # created at step 3.
                            
    ##### YOUR CODE HERE
    #print("printing df_agg")
    #print(df_agg)
    return df_agg



In [44]:
def test_subword_enrichment(func):
    """`func` should be an implementation of subword_enrichment as 
    defined above.
    """
    vocab = ["ABCD", "BCDA", "CDAB", "DABC"]
    df = pd.DataFrame([
        [1, 1, 2, 1],
        [3, 4, 2, 4],
        [0, 0, 1, 0],
        [1, 0, 0, 0]], index=vocab)
    expected = pd.DataFrame([
        [14, 14, 18, 14],
        [22, 26, 18, 26],
        [10, 10, 14, 10],
        [14, 10, 10, 10]], index=vocab)
    new_df = func(df, n=2)
    assert np.array_equal(expected.columns, new_df.columns), \
        "Columns are not the same"
    assert np.array_equal(expected.index, new_df.index), \
        "Indices are not the same"
    assert np.array_equal(expected.values, new_df.values), \
        "Co-occurrence values aren't the same"    

In [45]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_subword_enrichment(subword_enrichment)

### Your original system [3 points]

This question asks you to design your own model. You can of course include steps made above (ideally, the above questions informed your system design!), but your model should not be literally identical to any of the above models. Other ideas: retrofitting, autoencoders, GloVe, subword modeling, ... 

Requirements:

1. Your code must operate on one of the count matrices in `data/vsmdata`. You can choose which one. __Other pretrained vectors cannot be introduced__.

1. Your code must be self-contained, so that we can work with your model directly in your homework submission notebook. If your model depends on external data or other resources, please submit a ZIP archive containing these resources along with your submission.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [3]:
# Enter your system description in this cell.

"""
    The system builds of from the pipeline shown in the class
    i.e. ppmi->lsa->ae to:
    ppmi-> lsa->  subword_enrichment-> weighted_lsa_representation-> MMD-VAE-> Retrofitting
    
    
    I also tried using ppmi->lsa->subword enrichment, it gave
    poor results e.g:
                    finance    0.000000
                    balance    0.004174
                    chance     0.006348
                    advance    0.006587
                    glance     0.006653
    However a linear combination of ppmi and enrichment would allowus to incorporate more morphological
    information hence I used a weihting strategy:
    enriched_neighbours_ppmi_lsa_k = (w1*dataset_ppmi_lsa_k + w2*dataset_ppmi_lsa_k_ngram_enrichmet)/(w1+w2)
    This gave better resuts:
                            neighbours_enriched
                            finance        0.000000
                            banking        0.081684
                            investment     0.087688
                            investments    0.117994
                            management     0.121275
   
    
    The system then uses the above weighted representation to train an 
    MMD-VAE which tries to constrains the latent representation to unit Gaussian.
    I believe the advantage is that it should be robust to 
    different distance measures while also making the latent space more continous and meaninful
    
    
    After lsa and autoencoder we loose information because of the representation.
    So retrofitting is added to account for information loss and also it helps
    mitigating issues which could arise because of dataset not being sufficiently representative
                            
"""
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass


NameError: name 'os' is not defined

## Bake-off [1 point]

For the bake-off, we will release two additional datasets. The announcement will go out on the discussion forum. We will also release reader code for these datasets that you can paste into this notebook. You will evaluate your custom model $M$ (from the previous question) on these new datasets using `full_word_similarity_evaluation`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [46]:
# Enter your bake-off assessment code into this cell. 
# Please do not remove this comment.

if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    from collections import defaultdict
    from nltk.corpus import wordnet as wn
    import numpy as np
    import os
    import pandas as pd
    import retrofitting
    from retrofitting import Retrofitter
    import utils


    def get_wordnet_edges():
        edges = defaultdict(set)
        for ss in wn.all_synsets():
            lem_names = {lem.name() for lem in ss.lemmas()}
            for lem in lem_names:
                edges[lem] |= lem_names
        return edges

    def convert_edges_to_indices(edges, Q):
        lookup = dict(zip(Q.index, range(Q.shape[0])))
        index_edges = defaultdict(set)
        for start, finish_nodes in edges.items():
            s = lookup.get(start)
            if s:
                f = {lookup[n] for n in finish_nodes if n in lookup}
                if f:
                    index_edges[s] = f
        return index_edges

    from torch_autoencoder_mmd_info import TorchAutoencoderMMDIG
    model = TorchAutoencoderMMDIG()
    
    
    
    dataset_filename = "giga_window20-flat.csv.gz"
    k_dims = 100
    test_word = "finance"
    use_subword_enrichment = True
    
    dataset = pd.read_csv(
        os.path.join(VSM_HOME, dataset_filename), index_col=0)
    np_arr = dataset.to_numpy()
    print(len(np_arr))
    neighbours_raw = vsm.neighbors(test_word, dataset).head()
    print("neighbours_raw")
    print(neighbours_raw)
    
 
    dataset_ppmi = vsm.pmi(dataset, positive=True)
    #dataset_ppmi = vsm.pmi(enriched_model, positive=True)
    neighbours_ppmi = vsm.neighbors(test_word, dataset_ppmi).head()
    print("neighbours_ppmi")
    print(neighbours_ppmi)
    
    
    dataset_ppmi_lsa_k = vsm.lsa(dataset_ppmi, k=k_dims)
    neighbours_ppmi_lsa_k = vsm.neighbors(test_word, dataset_ppmi_lsa_k).head()
    print("neighbours_ppmi_lsa_k")
    print(neighbours_ppmi_lsa_k)
    enriched_neighbours_ppmi_lsa_k = dataset_ppmi_lsa_k
    if use_subword_enrichment:
        enriched_model = subword_enrichment(dataset_ppmi_lsa_k, n=4)
        w1 = 1
        w2 = 0.01
        w = w1+w2
        enriched_neighbours_ppmi_lsa_k = (w1*dataset_ppmi_lsa_k.to_numpy() + w2*enriched_model.to_numpy())/(w)
        enriched_neighbours_ppmi_lsa_k = pd.DataFrame(enriched_neighbours_ppmi_lsa_k, index=dataset.index)
        neighbours_enriched = vsm.neighbors(test_word, enriched_neighbours_ppmi_lsa_k).head()
        print("neighbours_enriched")
        print(neighbours_enriched)
        from torch_autoencoder_mmd_info import TorchAutoencoderMMDIG
        model = TorchAutoencoderMMDIG()
        df_train = pd.DataFrame(enriched_neighbours_ppmi_lsa_k, index=dataset.index)
        enriched_neighbours_ppmi_lsa_k_ae = model.fit(df_train)
        enriched_neighbours_ppmi_lsa_k_ae.to_pickle("enriched_neighbours_ppmi_lsa_k_ae")
        neighbours_enriched_ae = vsm.neighbors(test_word, enriched_neighbours_ppmi_lsa_k_ae).head()
        print("neighbours_enriched")
        print(neighbours_enriched_ae)
        #dataset_ppmi_lsa_k = enriched_neighbours_ppmi_lsa_k
        
    
    wn_edges = get_wordnet_edges()
    wn_index_edges = convert_edges_to_indices(wn_edges, enriched_neighbours_ppmi_lsa_k_ae)
    wn_retro = Retrofitter(verbose=True)
    X_retro = wn_retro.fit(enriched_neighbours_ppmi_lsa_k_ae, wn_index_edges)
    #print("X_retro")
    #print(X_retro)
    neighbours_retro = vsm.neighbors(test_word, X_retro).head()
    print("neighbours_retro")
    print(neighbours_retro)
    
    #model.fit(neighbours_ppmi_lsa_k)
    


5000
neighbours_raw
finance    0.000000
.          0.132519
</p>       0.139872
<p>        0.141332
said       0.158305
dtype: float64
neighbours_ppmi
finance     0.000000
banking     0.363871
monetary    0.404030
loans       0.413165
banks       0.415815
dtype: float64
neighbours_ppmi_lsa_k
finance     0.000000
banking     0.136505
monetary    0.167089
debt        0.171278
loans       0.172590
dtype: float64
neighbours_enriched
finance        0.000000
banking        0.081684
investment     0.087688
investments    0.117994
management     0.121275
dtype: float64


recons tensor(12.1201, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0091, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 12.129206657409668
recons tensor(11.7807, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0089, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 23.918808579444885
recons tensor(11.4150, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0092, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 35.34300649166107
recons tensor(10.9463, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0105, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 46.299816608428955
recons tensor(10.5697, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0122, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 56.881749629974365


Finished epoch 1 of 100; error is 10.58193302154541

recons tensor(10.2517, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0134, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 10.265105128288269
recons tensor(9.9949, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0144, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 20.27440059185028
recons tensor(9.6906, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0158, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 29.980785965919495
recons tensor(9.4602, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0163, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 39.457321643829346
recons tensor(9.7280, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0173, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 49.202592253685


Finished epoch 2 of 100; error is 9.745270729064941

recons tensor(9.1287, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0175, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 9.146275401115417
recons tensor(9.2568, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0177, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 18.420770287513733
recons tensor(8.8318, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0187, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 27.27120268344879
recons tensor(8.7020, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0188, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 35.99198365211487
recons tensor(8.0707, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0189, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 44.0815110206604


...

recons tensor(1.1230, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0032, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 1.12625253200531
recons tensor(1.1217, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0032, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 2.2511885166168213
recons tensor(1.1907, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0031, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 3.444967269897461
recons tensor(1.1321, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0036, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 4.580617547035217
recons tensor(1.1059, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0028, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 5.689290761947632


Finished epoch 99 of 100; error is 1.1086732149124146

recons tensor(1.0879, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0033, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 1.0911890268325806
recons tensor(1.1242, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0037, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 2.2190468311309814
recons tensor(1.1647, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0031, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 3.38686466217041
recons tensor(1.1413, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0032, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 4.531291365623474
recons tensor(1.1433, device='cuda:0', grad_fn=<MseLossBackward>)
mmd_err tensor(0.0034, device='cuda:0', grad_fn=<SubBackward0>)
epoch_error 5.678016543388367


Finished epoch 100 of 100; error is 1.1467251777648926

neighbours_enriched
finance        0.000000
investment     0.117755
banking        0.141329
investments    0.141360
fund           0.185637
dtype: float64


Converged at iteration 8; change was 0.0044 

neighbours_retro
finance        0.000000
investment     0.114872
banking        0.126206
investments    0.139357
interest       0.173808
dtype: float64


neighbours_enriched
finance        0.000000
investment     0.114872
banking        0.126206
investments    0.139357
monetary       0.182607
dtype: float64


In [ ]:
# On an otherwise blank line in this cell, please enter
# your "Macro-average" value as reported by the code above. 
# Please enter only a number between 0 and 1 inclusive.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE


